In [1]:
from IPython import display
from ipywidgets import widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import randint
import math
from pandas.tseries.offsets import MonthEnd, MonthBegin

class Cashflow_simulation:
    
    def __init__(self):
        style = {'description_width': 'initial'}
        self.marging_industry_text = widgets.Text(description='Margem:', placeholder="2%", value="2%", style=style)
        self.out = widgets.Output()
        display.display(self.marging_industry_text, self.out)
        
        
cf = Cashflow_simulation()

Text(value='2%', description='Margem:', placeholder='2%', style=DescriptionStyle(description_width='initial'))

Output()